In [1]:
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np


def get_data(input_path):
    all_imgs = []

    classes_count = {}

    class_mapping = {}

    img_class_mapping = {}

    visualise = False

    # not using vOC2007, 2012 only
    # data_paths = [os.path.join(input_path, s) for s in ['VOC2007', 'VOC2012']]
    data_paths = [os.path.join(input_path, s) for s in ['VOC2012']]

    print('Parsing annotation files')

    for data_path in data_paths:

        annot_path = os.path.join(data_path, 'Annotations')
        imgs_path = os.path.join(data_path, 'JPEGImages')
        imgsets_path_trainval = os.path.join(data_path, 'ImageSets', 'Main', 'trainval.txt')
        imgsets_path_test = os.path.join(data_path, 'ImageSets', 'Main', 'test.txt')

        trainval_files = []
        test_files = []
        try:
            with open(imgsets_path_trainval) as f:
                for line in f:
                    trainval_files.append(line.strip() + '.jpg')
        except Exception as e:
            print(e)

        try:
            with open(imgsets_path_test) as f:
                for line in f:
                    test_files.append(line.strip() + '.jpg')
        except Exception as e:
            if data_path[-7:] == 'VOC2012':
                # this is expected, most pascal voc distibutions dont have the test.txt file
                pass
            else:
                print(e)

        annots = [os.path.join(annot_path, s) for s in os.listdir(annot_path)]
        idx = 0
        for annot in annots:
            try:
                idx += 1

                et = ET.parse(annot)
                element = et.getroot()

                element_objs = element.findall('object')
                element_filename = element.find('filename').text
                element_width = int(element.find('size').find('width').text)
                element_height = int(element.find('size').find('height').text)

                if len(element_objs) > 0:
                    annotation_data = {'filepath': os.path.join(imgs_path, element_filename), 'width': element_width,
                                       'height': element_height, 'bboxes': []}

                    if element_filename in trainval_files:
                        annotation_data['imageset'] = 'trainval'
                    elif element_filename in test_files:
                        annotation_data['imageset'] = 'test'
                    else:
                        annotation_data['imageset'] = 'trainval'

                for element_obj in element_objs:
                    class_name = element_obj.find('name').text
                    if class_name not in classes_count:
                        classes_count[class_name] = 1
                    else:
                        classes_count[class_name] += 1

                    if class_name not in class_mapping:
                        class_mapping[class_name] = len(class_mapping)

                    obj_bbox = element_obj.find('bndbox')
                    x1 = int(round(float(obj_bbox.find('xmin').text)))
                    y1 = int(round(float(obj_bbox.find('ymin').text)))
                    x2 = int(round(float(obj_bbox.find('xmax').text)))
                    y2 = int(round(float(obj_bbox.find('ymax').text)))
                    difficulty = int(element_obj.find('difficult').text) == 1
                    annotation_data['bboxes'].append(
                        {'class': class_name, 'x1': x1, 'x2': x2, 'y1': y1, 'y2': y2, 'difficult': difficulty})
                all_imgs.append(annotation_data)

                img_class_mapping[element_filename] = class_name

                if visualise:
                    img = cv2.imread(annotation_data['filepath'])
                    for bbox in annotation_data['bboxes']:
                        cv2.rectangle(img, (bbox['x1'], bbox['y1']), (bbox[
                                                                          'x2'], bbox['y2']), (0, 0, 255))
                    cv2.imshow('img', img)
                    cv2.waitKey(0)

            except Exception as e:
                print(e)
                continue
    return all_imgs, classes_count, class_mapping, img_class_mapping

In [2]:
all_imgs, classes_count, class_mapping, img_class_mapping  = get_data("/Users/xuejiaxin/Dropbox/My Mac (Jiaxin的MacBook Pro)/Downloads/VOCdevkit")

Parsing annotation files


In [3]:
path_to_data = "/Users/xuejiaxin/Dropbox/My Mac (Jiaxin的MacBook Pro)/Documents/GitHub/pytorch-CycleGAN-and-pix2pix/datasets/PASCAL/Seg_image/"

path_to_car = "/Users/xuejiaxin/Dropbox/My Mac (Jiaxin的MacBook Pro)/Documents/GitHub/pytorch-CycleGAN-and-pix2pix/datasets/PASCAL/car/"

path_to_label = "/Users/xuejiaxin/Dropbox/My Mac (Jiaxin的MacBook Pro)/Documents/GitHub/pytorch-CycleGAN-and-pix2pix/datasets/PASCAL/Label_image/"

path_to_A = "/Users/xuejiaxin/Dropbox/My Mac (Jiaxin的MacBook Pro)/Documents/GitHub/pytorch-CycleGAN-and-pix2pix/datasets/PASCAL/car/A/"
path_to_B = "/Users/xuejiaxin/Dropbox/My Mac (Jiaxin的MacBook Pro)/Documents/GitHub/pytorch-CycleGAN-and-pix2pix/datasets/PASCAL/car/B/"

In [4]:
os.chdir(path_to_data)
file_names = os.listdir()

In [7]:
cars = []
for s in file_names:
    if img_class_mapping[s] == "car":
        cars.append(s)

len(cars)


166

In [8]:
from sklearn.model_selection import train_test_split
from shutil import copy

X_train, X_test, y_train, y_test = train_test_split(cars, cars, test_size=0.2, random_state=6, shuffle=True)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=6, shuffle=True)

print(X_train[0], y_train[0])
print(X_test[0], y_test[0])
print(X_val[0], y_val[0])
print(len(X_train), len(X_test), len(X_val))


for train in X_train:
    copy(path_to_data + train, path_to_A + "train")
    copy(path_to_label + train, path_to_B + "train")

for test in X_test:
    copy(path_to_data + test, path_to_A + "test")
    copy(path_to_label + test, path_to_B + "test")

for val in X_val:
    copy(path_to_data + val, path_to_A + "val")
    copy(path_to_label + val, path_to_B + "val")

2008_004552.jpg 2008_004552.jpg
2011_000239.jpg 2011_000239.jpg
2009_000825.jpg 2009_000825.jpg
132 17 17


In [ ]:
python datasets/combine_A_and_B.py --fold_A "./datasets/PASCAL/car/A" --fold_B "./datasets/PASCAL/car/B" --fold_AB "./datasets/PASCAL/pairs_car"

python train.py --dataroot ./datasets/PASCAL/pairs_car --name PASCAL_pix2pix --model pix2pix --direction AtoB --gpu_ids -1 --display_id 0